In [1]:
#ラベル付け
import json
import csv
import pandas as pd
import MeCab
import time

In [2]:
def readpdCSV(path, name, file_type):
    df = pd.read_csv(path+name+file_type, dtype={"body":"str"},encoding="utf-8")

    return df

In [3]:
def readJSON(path):
    data = json.load(open(path, 'r'))

    return data

In [5]:
def pickArticle(i, b, json_data):
    sep_body = []
    # print(len(h))
    # for i in range(len(h)):
    # 一文ごとに分割してからの特徴づけの場合
#     split_body = str(b).split("。")
#     del split_body[-1]
#     split_body = article_to_sentence(b)

#     sep_body.append(countNoun(h, json_data))
    for body, j in zip(b, range(len(b))):
        sep_body.append([i, j, body, countNoun(body, json_data)])

#     for sep in sep_body:
#         sep[3] = getWeight(sep[3], head, json_data)
    
    return sep_body

In [6]:
def countNoun(body, json_data):
    cat_set = ""
    body_sep = sepNoun(body)
    cat_data = {"keiki": "0", "bukka": "0", "kinri": "0", "money": "0", "boueki": "0", "koyou": "0", "kojin": "0", "kainyu": "0", "oil": "0", "kabu": "0", "saiken": "0", "youjin":"0", "seiji":"0"}
    
    for key in json_data.keys():
        if len(list(set(body_sep) & set(json_data[key]))) > 0:
            cat_data[key] = "1"
    
    for key in json_data.keys():
        cat_set += cat_data[key]

    return cat_set

In [7]:
def sepNoun(data):
    mt = MeCab.Tagger('mecabrc')
    mt.parse("")
    # print(data)
    node = mt.parseToNode(data)
    # print(node.surface)
    return getNouns(node)

In [8]:
def getNouns(node):
    words = []
    word_stack = ""
    while node:
        noun = node.feature.split(",")
        word = node.surface
        if(checkNoun(noun)):
            word_stack += word
        if(word_stack != "" and noun[0] != '名詞'):
            if(checkNum(word_stack)):
                words.append(word_stack)
            word_stack = ""
        node = node.next
    # words = " ".join(words)
    return words

In [9]:
def checkNoun(noun):
    zyogai = ["副詞可能", "非自立", "代名詞", "助詞類接続"]
    if(noun[0] == '名詞'):
        if(noun[1] in zyogai):
            return False
        else:
            return True
        
def checkNum(word_stack):
    mt = MeCab.Tagger()
    mt.parse("")

    node = mt.parseToNode(word_stack)
    while node:
        noun = node.feature.split(",")
        if(noun[1] == '数'):
            return False
        node = node.next

    return True

In [19]:
file_pass = "./reuters_csv/reuters_topNews_"
file_names = ["2008_2", "2009_2"]
file_type = ".csv"
sepfile_names = ["keiki", "bukka", "kinri", "money", "boueki", "koyou", "kojin", "kainyu", "oil", "kabu", "saiken", "sonota"]
sep_data = []
picked_data = []
# csvの読み込み
df1 = readpdCSV(file_pass, file_names[0], file_type)
df2 = readpdCSV(file_pass, file_names[1], file_type)
df = pd.concat([df1, df2], ignore_index=True)
json_data = readJSON("./json/word_dict.json")

In [25]:
for i, b in zip(df.index,df["body"]):
#     sep_body = b.split("。")
#     if(sep_body[-1] == ""):
#         del sep_body[-1]
#     for sep in sep_body:
#         if(picked_data == []):
#             picked_data.append(["parentid", "childid", "body", "label"])
        picked_data.append(pickArticle(i, b, json_data))

df["label"] = picked_data

In [26]:
label_data = [["parentid", "childid", "body", "label"]]
# print(picked_data[1])
for data in picked_data:
    for row in data:
#         print(row)
#         time.sleep(1.0)
        label_data.append(row)

# print(label_data[2][2])
# for(data in picked_data):
#     print(data[3])

In [27]:
with open("./reuters_article00.csv", "w") as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerows(label_data)